In [1]:
import os, sys; sys.path.append(os.path.abspath('../'))

In [2]:
# Crawl the data directory
import os

data_dir = r'..\data\baseline'

data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.mat')]
# loop over the files

import scipy.io

All_network_X = []
All_network_y = []

for f in data_files[:10]:
    data = scipy.io.loadmat(f)
    test_x = data['test_x']
    test_out = data['test_out']

    All_network_X.append(test_x)
    All_network_y.append(test_out)

import numpy as np

All_network_X = np.array(All_network_X)
All_network_y = np.array(All_network_y)

In [3]:
import os, sys; sys.path.append(os.path.abspath('../'))
# from inspector.rsa import ESM

In [4]:
from typing import Tuple
import numpy.typing as npt
import numpy as np

from inspector.rsa.netrep_wrapper import NetRep
import numpy as np
from inspector.utility import plot_distmat

from netrep.metrics.stochastic import GaussianStochasticMetric

class GSM_tests(NetRep):
    def __init__(self, verbose = True):
        super().__init__(verbose)

    def get_means_and_covs(self, X, y):
        """Helper method that computes class-conditional means and covariances."""
        
        classes = np.unique(y)
        means = np.stack([np.mean(X[y==labels], 0) for labels in classes], 0) # Meaning: compute the mean of each class
        covs = np.stack([np.cov(X[y==labels].T) for labels in classes], 0) # Meaning: compute the covariance of each class
        return means, covs

    def digest(self):

        # check if the data has already been digested
        if self.data["digested_flag"]:
            if self.verbose: print("Data already digested, skipping digestion")
            return "TBD Digestion Report"

        X_all = self.data['X']
        Y_all = self.data['y']

        self.data['digested_networks'] = []

        for i in range(len(X_all)):
            print(X_all[i].shape, Y_all[i].shape)
            means, covs = self.get_means_and_covs(X_all[i], Y_all[i])
            print(means.shape, covs.shape)
            # print(means, covs)
            self.data['digested_networks'].append((means, covs))

        if self.verbose: print(f"Successfully digested data, available in self.data['digested_networks']")

        # set the digested flag
        self.data["digested_flag"] = True
        
        # return the report of digested networks
        return "TBD Digestion Report"

    def score(self):
        metric = GaussianStochasticMetric(1)
        return super().score(metric)
    
    def plot(self, title="Gaussian Stochastic Metric", out_path=None):
        '''Plot the RSA score'''
        plot_distmat(self.data['distance_matrix'], title=title, out_path=out_path)

In [5]:
y_last_50_avg = All_network_y[:, :, :, -50:]
y_last_50_avg.shape
y_last_50_avg = np.average(y_last_50_avg, axis=3)
y_last_50_avg.shape
y_last_50_avg = y_last_50_avg.reshape(y_last_50_avg.shape[0], -1)
y_last_50_avg.shape

# for every value in y_last_50_avg, if it is greater than 0, set it to 1, else set it to -1
y_last_50_avg[y_last_50_avg > 0] = 1
y_last_50_avg[y_last_50_avg < 0] = -1

x_nuron_avg = np.average(All_network_X, axis=3)
x_nuron_avg.shape
x_nuron_avg = x_nuron_avg.reshape(10, 4*50, 300)
x_nuron_avg.shape

(10, 200, 300)

In [6]:
x_nuron_avg += 100

In [7]:
def digest_and_score(X, y):
    netrep_metric = GSM_tests(verbose = True)
    ingestion_report = netrep_metric.ingest(X, y, preprcessed = False)
    digestion_report = netrep_metric.digest()
    energy_distance = netrep_metric.score()
    netrep_metric.plot()
    return energy_distance

In [8]:
lim = 2
energy_distance = digest_and_score(x_nuron_avg[:lim], y_last_50_avg[:lim])

(200, 300) (200,)
(2, 300) (2, 300, 300)
(200, 300) (200,)
(2, 300) (2, 300, 300)
Successfully digested data, available in self.data['digested_networks']


  0%|          | 0/1 [00:00<?, ?it/s]c:\users\monac\documents\github\netrep\netrep\metrics\stochastic.py:366: RuntimeWarning: invalid value encountered in sqrt
  sX = np.einsum("ijk,ik,ilk->ijl", uX, np.sqrt(vX), uX)
c:\users\monac\documents\github\netrep\netrep\metrics\stochastic.py:369: RuntimeWarning: invalid value encountered in sqrt
  sY = np.einsum("ijk,ik,ilk->ijl", uY, np.sqrt(vY), uY)


ValueError: array must not contain infs or NaNs

In [9]:
a = []
netrep_metric = GSM_tests(verbose = True)

for i in range(len(x_nuron_avg)):
    print(x_nuron_avg[i].shape, x_nuron_avg[i].shape)
    means, covs = netrep_metric.get_means_and_covs(x_nuron_avg[i], y_last_50_avg[i])
    a.append((means, covs))

(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)
(200, 300) (200, 300)


In [12]:
metric = GaussianStochasticMetric(1)

In [ ]:
metric.fit(a[0], a[1])
distmat_w2[i,j] = metric.score(a[0], a[1])

In [17]:
# compute the class-conditional means and covariances for each network
means_x1, covs_x1 = a[0][0], a[0][1]
means_x2, covs_x2 = a[1][0], a[1][1]
means_x3, covs_x3 = a[2][0], a[2][1]

print("(means_x1.shape, covs_x2.shape)", (means_x1.shape, covs_x1.shape))

# compile into mu and sigma dicts for easier processing
means = (means_x1, means_x2, means_x3)
covs = (covs_x1, covs_x2, covs_x3)

(means_x1.shape, covs_x2.shape) ((2, 300), (2, 300, 300))


In [18]:
distmats_w2 = dict()
import itertools

# compute Wasserstein ground metric for alpha=(0, 1, 2)
for alpha in (0, 1, 2):
    distmat_w2 = np.zeros((3, 3))
    metric = GaussianStochasticMetric(alpha)
    for i,j in itertools.combinations(range(3), 2): # Meaning: compute the distance between each pair of networks
        
        # the fit method expects a list [means, covs]
        Xi = (means[i], covs[i]) # Meaning: Xi is a tuple of the means and covariances of network i
        Xj = (means[j], covs[j]) # Meaning: Xj is a tuple of the means and covariances of network j


        metric.fit(Xi, Xj)
        distmat_w2[i,j] = metric.score(Xi, Xj)

        # My score printing code
        print(f"alpha {alpha}: Network {i+1} vs Network {j+1} -> score {distmat_w2[i,j]}") # Meaning: print the pair of networks

    # dist matrix is symmetric
    distmat_w2 += distmat_w2.T
    distmats_w2[alpha] = distmat_w2

ValueError: array must not contain infs or NaNs

In [ ]:
a[0][1]

In [ ]:
x_nuron_avg.shape, y_last_50_avg.shape

In [ ]:
X, y = x_nuron_avg[0], y_last_50_avg[0]

classes = np.unique(y)

In [ ]:
K = len(classes)
print(classes, K)

In [ ]:
means = np.stack([np.mean(X[y==label], 0) for label in classes], 0) 

In [ ]:
# covs = np.stack([np.cov(X[y==label].T) for label in classes], 0) # Meaning: compute the covariance of each class
covs = np.stack([np.cov(X[y==labels].T) for labels in classes], 0)

In [ ]:
covs

In [ ]:
covs